<a href="https://colab.research.google.com/github/NatLibFi/Annif/blob/kvp2019-workshop/arviopohjien_luonti_juho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install gspread-pandas

In [0]:
! mkdir -p  /root/.config/gspread_pandas/
! echo '{"installed":{"client_id":"257572009167-uai362klp88kr3hluabtoujf2hnd7fr6.apps.googleusercontent.com","project_id":"formal-archive-254810","auth_uri":"https://accounts.google.com/o/oauth2/auth","token_uri":"https://oauth2.googleapis.com/token","auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs","client_secret":"x554t-fVXmFDrENKrXu8GuKy","redirect_uris":["urn:ietf:wg:oauth:2.0:oob","http://localhost"]}}' > /root/.config/gspread_pandas/google_secret.json

In [0]:
import pandas as pd
import random
from gspread_pandas import Spread, Client

In [178]:
# Fetch kuvailut

client = Client()
spreads = client.find_spreadsheet_files_in_folders('Dokumenttien kuvailut')

df_kuvailut1 = pd.DataFrame()
df_kuvailut2 = pd.DataFrame()

for kuvailu_id, spread in enumerate(spreads['Dokumenttien kuvailut']):
  print(spread['name'])
  # Tietokirjat:
  if spread['name'].startswith("kuvailut_tieto"):
      df_sheet = Spread(spread['id'], sheet=0).sheet_to_df(index=None)
      df_sheet['kid'] = kuvailu_id
      df_sheet['tiedostonimi'] = spread['name']
      df_sheet.rename(columns={'isbn': 'url kokotekstiin'}, inplace=True)
      df_kuvailut1 = df_kuvailut1.append(df_sheet) #[cols_to_keep])
  else:
      df_sheet = Spread(spread['id'], sheet=0).sheet_to_df(index=None)
      df_sheet['kid'] = kuvailu_id
      df_sheet['tiedostonimi'] = spread['name']
      df_sheet.columns = df_sheet.columns.str.lower()
      df_kuvailut2 = df_kuvailut2.append(df_sheet) #[cols_to_keep])    
  

Copy of Kuvailut Fennica
jyx_kuvailut Mona
kuvailut_tieto Osma
Copy of Kuvailut K07
Copy of Kuvailut K16
Copy of Kuvailut K31
kuvailut_tieto Mona
kuvailut_tieto Fennica
jyx_kuvailut Osma
kuvailut_tieto Juho
jyx_kuvailut Juho
jyx_kuvailut_orig


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


SKT4 VL Copy of Kuvailut 
SKT3 JM Kuvailut Copy of 
 SKT1 KL Copy of Kuvailut
SKT2 PM Copy of Kuvailut 
Copy of Kuvailut K02
Copy of Kuvailut K17
Copy of Kuvailut K05
Copy of Kuvailut K14
Copy of Kuvailut K11
Copy of Kuvailut K08
Copy of Kuvailut K32
Copy of Kuvailut K23
Copy of Kuvailut K29
Copy of Kuvailut K20
Copy of Kuvailut K26
Copy of Kuvailut K19
Copy of Kuvailut K10
Copy of Kuvailut K38
Copy of Kuvailut K13
Copy of Kuvailut K35
Copy of Kuvailut K40
Copy of Kuvailut K28
Copy of Kuvailut K25
Copy of Kuvailut K37
Copy of Kuvailut K22
Copy of Kuvailut K18
Copy of Kuvailut K06
Copy of Kuvailut K12
Copy of Kuvailut K03
Copy of Kuvailut K09
Copy of Kuvailut K33
Copy of Kuvailut K39
Copy of Kuvailut K21
Copy of Kuvailut K36
Copy of Kuvailut K04
Copy of Kuvailut K01
Copy of Kuvailut K34
Copy of Kuvailut K24
Copy of Kuvailut K15
Copy of Kuvailut K30
Copy of Kuvailut K27


In [179]:
# Preprocess 1

df_kuvailut = pd.concat([df_kuvailut1, df_kuvailut2])
cols_to_keep = ['id', 'nimeke', 'url kokotekstiin', 'aiheet', 'kid', 'tiedostonimi']
df_kuvailut = df_kuvailut[cols_to_keep]
df_kuvailut.rename(columns={'id': 'did'}, inplace=True)
df_kuvailut[['did', 'nimeke', 'url kokotekstiin']] = df_kuvailut[['did', 'nimeke', 'url kokotekstiin']].mask(df_kuvailut[['did', 'nimeke', 'url kokotekstiin']]=='', None).ffill()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [0]:
# Set types of index source method
ihminen = ['kuvailut_tieto Mona', 'jyx_kuvailut Mona', 'Copy of Kuvailut K']
annif_avusteiset = ['kuvailut_tieto Osma', 'kuvailut_tieto Juho', 'jyx_kuvailut_orig']
#annif = ['']
fennica = ['kuvailut_tieto Fennica', 'Copy of Kuvailut Fennica']

df_kuvailut['tyyppi'] = df_kuvailut['tiedostonimi'].apply(lambda x: 'annif_avusteinen' if x in annif_avusteiset
                                                                  else ('fennica' if x in fennica else 'ihminen'
                                                                  ))

df_kuvailut = df_kuvailut[df_kuvailut.did != 'D00']  # Poista esimerkkikuvailu

In [0]:
# Create header and footer to insert around kuvailu 

df_alaosa0 = pd.DataFrame({'did': [''], 'nimeke': [''], 'url kokotekstiin': [''], 'aiheet': [''], 'kid': ['']})
df_alaosa1 = pd.DataFrame({'did': [''], 'nimeke': [''], 'url kokotekstiin': [''], 'aiheet': ['arvosana'], 'kid': ['']})
df_alaosa2 = pd.DataFrame({'did': ['']*4, 'nimeke': ['']*4, 'url kokotekstiin': ['Kuvailu on kattava', 'Käytetyt asiasanat ovat sisällön kannalta merkityksellisiä', 'Kuvailu on laadukas', 'Vapaa kenttä'], 'aiheet': ['']*4, 'kid': ['']*4})

df_alaosa = pd.concat([df_alaosa0, df_alaosa1, df_alaosa2, df_alaosa0, df_alaosa0])

df_ylaosa = pd.DataFrame({'did': [''], 'kid': [''], 'nimeke': [''], 'url kokotekstiin': [''], 'aiheet': ['aiheet'], 'poistettavat': ['poista aihe merkitsemällä x']})


In [185]:
# Create files and sheets

min_num_aiheet = 1
max_num_ihmiskuvailut = 3
num_arvioija = 2
num_sheets = 10

include_types = ['fennica', 'annif_avusteinen']
columns = ['did', 'kid', 'nimeke', 'url kokotekstiin', 'aiheet'] #, 'tiedostonimi']

dokumentit = [(did, df_dokumentti) for did, df_dokumentti in df_kuvailut.groupby('did')]

for arvioija_ind in range(num_arvioija):
    print('arvioija_ind: ', arvioija_ind)
    spread = Spread('arvioija_' + str(arvioija_ind) , create_spread=True)
    spread.move('Test folder')
    random.shuffle(dokumentit)

    cnt_sheet = 0
    for did, df_dokumentti in dokumentit:
        spread.open_sheet(did, create=True)
        df_pohja = pd.DataFrame(columns=columns)
        print('dockument_id: ', did)
        kuvailut = [df_dokumentti_kuvailu 
                    for _, df_dokumentti_kuvailu 
                    in df_dokumentti.groupby('kid')
                    if df_dokumentti_kuvailu.tyyppi.iloc[0] in include_types
                    and len(df_dokumentti_kuvailu.aiheet[df_dokumentti_kuvailu.aiheet != '']) >= min_num_aiheet
                    ]
        ihmiskuvailut = [df_dokumentti_kuvailu 
                    for _, df_dokumentti_kuvailu 
                    in df_dokumentti.groupby('kid')
                    if df_dokumentti_kuvailu.tyyppi.iloc[0] == 'ihminen'
                    and len(df_dokumentti_kuvailu.aiheet[df_dokumentti_kuvailu.aiheet != '']) >= min_num_aiheet
                    ]
        random.shuffle(ihmiskuvailut)
        ihmiskuvailut = ihmiskuvailut[:max_num_ihmiskuvailut]
        kuvailut.extend(ihmiskuvailut)
        random.shuffle(kuvailut)
        
        #df_otsake = pd.DataFrame(kuvailut[0].iloc[:2])
        #spread.df_to_sheet(df_otsake, index=False)
        
        for df_dokumentti_kuvailu in kuvailut:
            df_pohja = df_pohja.append(df_ylaosa.append(df_dokumentti_kuvailu[columns], sort=False).append(df_alaosa, sort=False).fillna(''), sort=False)

        spread.df_to_sheet(df_pohja, index=False, headers=False)        
        cnt_sheet += 1
        if cnt_sheet > num_sheets:
            break


arvioija_ind:  0
dockument_id:  SKT10
dockument_id:  D07
dockument_id:  SKT17
dockument_id:  D09
dockument_id:  SKT40
dockument_id:  SKT08
dockument_id:  TI03
dockument_id:  SKT41
dockument_id:  SKT36
dockument_id:  SKT38
dockument_id:  TI10
arvioija_ind:  1
dockument_id:  TI06
dockument_id:  D10
dockument_id:  SKT47
dockument_id:  D17
dockument_id:  JYX05
dockument_id:  SKT43
dockument_id:  SKT05
dockument_id:  JYX06
dockument_id:  SKT41
dockument_id:  SKT42
dockument_id:  SKT44


0

In [0]:
# create a blank spreadsheet with a specified title.

spreadsheet = {
    'properties': {
        'title': title
    }
}
spreadsheet = service.spreadsheets().create(body=spreadsheet,
                                    fields='spreadsheetId').execute()
print('Spreadsheet ID: {0}'.format(spreadsheet.get('spreadsheetId')))


#To write to a sheet, you will need the spreadsheet ID, the range(s) in A1 notation,
#and the data you wish to write arranged in an appropriate request body object.
values = [
    [
        # Cell values ...
    ],
    # Additional rows
]
data = [
    {
        'range': range_name,
        'values': values
    },
    # Additional ranges to update ...
]
body = {
    'valueInputOption': value_input_option,
    'data': data
}
result = service.spreadsheets().values().batchUpdate(
    spreadsheetId=spreadsheet_id, body=body).execute()
print('{0} cells updated.'.format(result.get('totalUpdatedCells')))


#copy sheets
from googleapiclient import discovery

# TODO: Change placeholder below to generate authentication credentials. See
# https://developers.google.com/sheets/quickstart/python#step_3_set_up_the_sample
#
# Authorize using one of the following scopes:
#     'https://www.googleapis.com/auth/drive'
#     'https://www.googleapis.com/auth/drive.file'
#     'https://www.googleapis.com/auth/spreadsheets'
credentials = None

service = discovery.build('sheets', 'v4', credentials=credentials)

# The ID of the spreadsheet containing the sheet to copy.
spreadsheet_id = 'my-spreadsheet-id'  # TODO: Update placeholder value.

# The ID of the sheet to copy.
sheet_id = 0  # TODO: Update placeholder value.

copy_sheet_to_another_spreadsheet_request_body = {
    # The ID of the spreadsheet to copy the sheet to.
    'destination_spreadsheet_id': '',  # TODO: Update placeholder value.

    # TODO: Add desired entries to the request body.
}

request = service.spreadsheets().sheets().copyTo(spreadsheetId=spreadsheet_id, sheetId=sheet_id, body=copy_sheet_to_another_spreadsheet_request_body)
response = request.execute()

# TODO: Change code below to process the `response` dict:


In [0]:
#using gspread
scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive'
]

# making spreadsheets
create(title)

t == ""
for i in range(50):
  t == t+str(i)
  create()

